In [28]:
%load_ext autoreload
%autoreload 2

import torch
from src.architectures.helpers import CNNBlock, SqueezeExcitationBlock

from torchinfo import summary

img = torch.rand(1, 3, 224, 224)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
cnn_block = CNNBlock(3, 12, 3, 1, 0)

se_conv = SqueezeExcitationBlock(
    channels = 12,
    block=cnn_block,
    reduction_ratio=3
)

In [36]:
se_conv(img)

tensor([[[[-3.4180e-01,  3.2931e-01, -6.7161e-01,  ...,  1.9962e-01,
           -3.3435e-02,  2.2320e-01],
          [-5.1948e-01,  1.5836e-01, -1.7645e-01,  ...,  6.5331e-01,
            2.3412e-01, -4.4200e-02],
          [ 2.9854e-01, -6.3723e-01,  7.6901e-01,  ...,  1.9690e-01,
           -5.9790e-01, -6.6897e-01],
          ...,
          [ 3.1003e-01, -5.3229e-01,  2.3585e-02,  ..., -5.4026e-01,
           -1.6451e-01,  3.5951e-01],
          [-6.7161e-01, -3.1952e-01, -1.2306e-01,  ..., -6.7161e-01,
            4.3732e-02,  1.8555e-01],
          [-6.5144e-01, -5.6744e-01,  5.4109e-01,  ..., -1.1315e-01,
            8.9560e-01,  8.3474e-01]],

         [[-3.4286e-01,  4.1478e-02,  4.3695e-01,  ..., -3.5954e-01,
           -3.5954e-01, -2.1781e-01],
          [-3.5954e-01, -3.5954e-01, -3.5954e-01,  ..., -1.0229e-01,
           -3.5954e-01, -3.5954e-01],
          [ 4.7661e-01,  2.0864e-01, -1.5757e-01,  ...,  1.6391e-01,
            1.1190e+00, -3.5954e-01],
          ...,
     

In [41]:
a = torch.Tensor([
    [1, 2, 3, 4],
    [4, 5, 6, 7],
    [7, 8, 9, 10]
])

In [42]:
b = torch.Tensor([2, 2, 1])

In [44]:
a.shape, b.shape

(torch.Size([3, 4]), torch.Size([3]))

In [46]:
a*b.unsqueeze(-1)

tensor([[ 2.,  4.,  6.,  8.],
        [ 8., 10., 12., 14.],
        [ 7.,  8.,  9., 10.]])

In [25]:
net = SqueezeNet(freq=3)

3 [False, True, False, False, True, False, False, True] [96, 128, 128, 128, 256, 256, 256, 384, 384]


In [26]:
net.fb_in_channels

[96, 128, 128, 128, 256, 256, 256, 384, 384]

In [27]:
out = net(img)

conv1 torch.Size([1, 96, 109, 109])
maxpool1 torch.Size([1, 96, 54, 54])
fire2 torch.Size([1, 128, 54, 54])
fire3 torch.Size([1, 128, 54, 54])
fire4 torch.Size([1, 128, 54, 54])
maxpool4 torch.Size([1, 128, 26, 26])
fire5 torch.Size([1, 256, 26, 26])
fire6 torch.Size([1, 256, 26, 26])
fire7 torch.Size([1, 256, 26, 26])
fire8 torch.Size([1, 384, 26, 26])
maxpool8 torch.Size([1, 384, 12, 12])
fire9 torch.Size([1, 384, 12, 12])
dropout9 torch.Size([1, 384, 12, 12])


In [6]:
# summary(net, input_data=img)